In [1]:
# load package
# string 
import re

# math
import pandas as pd
import numpy as np
import scipy as sp
import random

# sys
import sys
import os
import time
import warnings

# nlp
from pattern.de import parse, conjugate, singularize, pluralize

# machine learning
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, gaussian_process, discriminant_analysis
from xgboost import XGBClassifier

# model utils
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection 
from sklearn import model_selection
from sklearn import metrics

# plot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix #??
# = show plots in Jupyter Notebook browser
%matplotlib inline 
mpl.style.use('ggplot') #??
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8 #??

# show all columns
from IPython.display import display
pd.options.display.max_columns = None

# self define
sys.path.append('../../utils/')

In [2]:
# read files
lb_wsp_2014 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2014.csv', sep = ';')
lb_wsp_2015 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2015.csv', sep = ';')
lb_wsp_2016 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2016.csv', sep = ';')
lb_wsp_2017 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2017.csv', sep = ';')
lb_wsp_2018 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2018.csv', sep = ';')
std_wsp_2014 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2014.csv', sep = ';')
std_wsp_2015 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2015.csv', sep = ';')
std_wsp_2016 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2016.csv', sep = ';')
std_wsp_2017 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2017.csv', sep = ';')
std_wsp_2018 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2018.csv', sep = ';')
# cat
d1 = lb_wsp_2014.copy()
d2 = lb_wsp_2015.copy()
d3 = lb_wsp_2016.copy()
d4 = lb_wsp_2017.copy()
d5 = lb_wsp_2018.copy()
#data1 = pd.concat([d1,d2,d3,d4], 0)
d6 = std_wsp_2014.copy()
d7 = std_wsp_2015.copy()
d8 = std_wsp_2016.copy()
d9 = std_wsp_2017.copy()
d10 = std_wsp_2018.copy()
#data2 = pd.concat([d6,d7,d8,d9,d10], 0)
#data = pd.concat([data1, data2], 0)

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# for training we use data from 14 - 17 in lb_wsp
train = pd.concat([d1,d2,d3,d4], 0)
test = d5
data = [train, test]

In [4]:
df = lb_wsp_2014[['Auftragsnummer', 'Beschreibung', 'Fahrgestellnummer']]

In [7]:
# 初衷是通过一个个的账单，统计每个属性，对应属性项在一段时间内出现的次数

def toFreqTable(df, att, date, clean = True):
    """
    input:
        df, DataFrame:
            the dataframe
        att, string:
            the column name of the target attribute
        date, string:
            the column name of the date attribute
    """
    # extract the att and date columns
    df = df[[att, date]]
    
    # change string date to date type
    # detect outlier
    fake = df[date].map(check_date)
    # if clean is True, drop the fake data
    if clean:
        print("Fake date exist, drop these dates directly")
        df = df.drop(df[fake].index)
    df[date] = pd.to_datetime(df[date])
    # add new column named 'count'
    df['count'] = pd.Series([1 for i in range(len(df))])
    # group and sum 
    df_g = df.groupby([date, att], as_index = False).agg({'count': 'sum'})
    return df_g

def check_date(s, sep = '.'):
    """
    only check the date grobly,
    set the fake date to True and the right date to false
    """
    items = s.split('.')
    #print(items)
    if len(items) <3:
        return True
    day = int(items[0])
    month = int(items[1])
    year = int(items[2])
    if day <= 31 and day > 0 and month <= 12 and month > 0 and year > 0:
        return False
    else:
        return True

In [8]:
av_ref = toFreqTable(train, 'Adressanredecode', 'Auftragsdatum')

Fake date exist, drop these dates directly


In [9]:
av_ref

,Auftragsdatum,Adressanredecode,count
0,2013-12-30,Firma,2
1,2014-01-02,Firma,95
2,2014-01-02,Frau,14
3,2014-01-02,Herr,19
4,2014-01-03,Firma,4
5,2014-01-03,Herr,74
6,2014-01-03,OHNE,2
7,2014-01-04,AN,15
8,2014-01-04,FA_F,22
9,2014-01-04,Firma,525


In [11]:
av_ref['Adressanredecode'].value_counts()

Herr          1219
Firma         1201
Frau          1170
AN             210
FA_F           169
FA_M            37
FR.DR.          23
HR.DR.          20
OHNE            17
HERR+FRAU        9
AUTOHAUS         4
FAHRSCHULE       3
HR U. FR         1
FR.PROF.         1
Name: Adressanredecode, dtype: int64

In [13]:
av_ref.to_csv('/Users/ihuangyiran/Desktop/tmp.csv', header= True, sep= ';')